In [4]:
import os

dir = r"C:\Users\maxma\OneDrive\Desktop\Shared Fiber Pho\FiberPhoStuff\data"
bpa = []
for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(".tev"):
                  bpa.append(root)

In [5]:
bpa

['C:\\Users\\maxma\\OneDrive\\Desktop\\Shared Fiber Pho\\FiberPhoStuff\\data\\Vision02-240222-112005',
 'C:\\Users\\maxma\\OneDrive\\Desktop\\Shared Fiber Pho\\FiberPhoStuff\\data\\Vision02-240222-135600']

In [ ]:
# raw signal from grbda and iso channels 
TRANGE = [-10, 30] # window size [start time relative to epoc onset, window duration]
BASELINE_PER = [-10, -6] # baseline period within our window
ARTIFACT = float("inf")
ymin = -7
ymax = 7
fig = plt.figure(figsize=(9, 20))
# increase spacing between subplots
fig.subplots_adjust(hspace=0.7)
for idx, event_code in enumerate([37,34]):
    trs = get_time_ranges(prtb, values=[event_code])
    filtered_csp_trs = modify_time_ranges(prtb, TRANGE, trs)
    filtered_grbda = get_filtered_stream_data(grbda, filtered_csp_trs)
    filtered_iso = get_filtered_stream_data(iso, filtered_csp_trs)
    min1 = np.min([np.size(x) for x in filtered_grbda])
    min2 = np.min([np.size(x) for x in filtered_iso])
    # filtered_grbda = [x[1:min1] for x in filtered_grbda]
    # filtered_iso = [x[1:min2] for x in filtered_iso]

    # Downsample and average 10x via a moving window mean
    N = 10 # Average every 10 samples into 1 value
    F405 = []
    F465 = []
    for lst in filtered_iso:
        small_lst = []
        for i in range(0, min2, N):
            small_lst.append(np.mean(lst[i:i+N-1])) # This is the moving window mean
        F405.append(small_lst)

    for lst in filtered_grbda: 
        small_lst = []
        for i in range(0, min1, N):
            small_lst.append(np.mean(lst[i:i+N-1]))
        F465.append(small_lst)


    #Create a mean signal, standard error of signal, and DC offset
    meanF405 = np.mean(F405, axis=0)
    stdF405 = np.std(F405, axis=0)/np.sqrt(len(filtered_iso))
    dcF405 = np.mean(meanF405)
    meanF465 = np.mean(F465, axis=0)
    stdF465 = np.std(F465, axis=0)/np.sqrt(len(filtered_grbda))
    dcF465 = np.mean(meanF465)

    # Create the time vector for each stream store
    ts1 = TRANGE[0] + np.linspace(1, len(meanF465), len(meanF465))/grbda.fs*N
    ts2 = TRANGE[0] + np.linspace(1, len(meanF405), len(meanF405))/iso.fs*N

    # Subtract DC offset to get signals on top of one another
    meanF405 = meanF405 - dcF405
    meanF465 = meanF465 - dcF465

    # Start making a figure with 4 subplots
    ax0 = fig.add_subplot(5, 1, idx+1)

    # Plotting the traces
    p1, = ax0.plot(ts1, meanF465, linewidth=2, color='green', label='GrbDA')
    p2, = ax0.plot(ts2, meanF405, linewidth=2, color='blueviolet', label='ISO')

    # Plotting standard error bands
    p3 = ax0.fill_between(ts1, meanF465+stdF465, meanF465-stdF465,
                          facecolor='green', alpha=0.2)
    #p4 = ax0.fill_between(ts2, meanF405+stdF405, meanF405-stdF405,
     #                     facecolor='blueviolet', alpha=0.2)

    # Plotting a line at t = 0 and t = 10
    p5 = ax0.axvline(x=0, linewidth=3, color='slategray', label='Cue Onset')
    p6 = ax0.axvline(x=10, linewidth=3, color='lightblue', label='Reward Delivery')

    # Finish up the plot
    ax0.set_xlabel('Seconds')
    ax0.set_ylabel('Signal (mV)')
    ax0.set_title('GFPG vs ISOS')
    ax0.legend(handles=[p1, p2, p5, p6], loc='upper right')
    # to set range based on minimum and maximum values in sample. created graphs with huge ranges
    # ax0.set_ylim(min(np.min(meanF465-stdF465), np.min(meanF405-stdF405)),
    #             max(np.max(meanF465+stdF465), np.max(meanF405+stdF405)))
    ax0.set_ylim(ymin,
                 ymax)
    ax0.set_xlim(TRANGE[0], TRANGE[1]+TRANGE[0])